In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
!pwd
import sys
import os
sys.path.append(os.path.abspath('../'))
import pandas as pd
from datetime import datetime

from data_helpers.syntax_graph import SyntaxGraph
from data_helpers.conllu_reader import CoNNLUReader

/Users/rabauti/repos/eki/prg1978__opikulausetekorpus/heete_loendid


# Tabel verb + ma andmete kogumine


**Ülesande püstitus**
    
Kõik laused, kus esineb tabelis [list_ma.csv](list_ma.csv) olev verb ja verbil on otsene alluv deprel=xcomp, VerbForm=Sup

**Tulemus** 

Tabel veergudega:
1. leitud lause, 
2. milline tabelis olevates verbidest seal esineb (algvormis), 
3. ma-infinitiivi vormis oleva verbi lemma, 
4. keeletase.
   

In [7]:
%%time

# verbid etteantud nimekirjast
df_verbs = pd.read_csv('list_ma.csv')
my_verbs = list(df_verbs['lemma'].unique())


file_name = '../data/Eesti-keele-kui-teise-keele-kooliõpikute-lausete-korpus-2021.conllu'
my_reader = CoNNLUReader(
   file_name = file_name
)


CPU times: user 996 μs, sys: 2.16 ms, total: 3.15 ms
Wall time: 3.34 ms


In [8]:
my_verbs

['kippuma',
 'tikkuma',
 'tükkima',
 'kibelema',
 'sügelema',
 'kihelema',
 'pürgima',
 'püüdlema',
 'pretendeerima',
 'kalduma',
 'tahtma',
 'valmistuma',
 'ehtima',
 'valmistama',
 'sättima',
 'asutama',
 'kohmitsema',
 'õppima',
 'harjutama',
 'häälestuma',
 'spetsialiseeruma',
 'arenema',
 'divergeeruma',
 'harjuma',
 'sobima',
 'kõlbama',
 'kvalifitseeruma',
 'saama',
 'pääsema',
 'vabanema',
 'jõudma',
 'mahtuma',
 'ulatuma',
 'ulatama',
 'küündima',
 'nõustuma',
 'soostuma',
 'kohustuma',
 'vaevuma',
 'painduma',
 'alanduma',
 'tegema',
 'pidama',
 'juhtuma',
 'sattuma',
 'trehvama',
 'sündima',
 'registreeruma',
 'innustuma',
 'keskenduma',
 'arvama',
 'tajuma',
 'riskima',
 'tõotama',
 'üritama',
 'piirduma']

In [9]:
%%time

date_time = datetime.now().strftime("%Y%m%d-%H%M%S")

collected_data = []
count = 0
for collection_id, graph in my_reader.get_sentences():
    # matrix for node distances
    dpath = graph.get_distances_matrix()
    
    # verb nodes
    verb_nodes = [v for v in graph.get_nodes_by_attributes(attrname="POS", attrvalue="VERB") if graph.nodes[v]["lemma"] in my_verbs]
    if not len(verb_nodes): continue
    
    # xcomp
    xcomp_nodes = graph.get_nodes_by_attributes(attrname="deprel", attrvalue="xcomp")
   
    if not len(xcomp_nodes): continue
    
    for verb in verb_nodes:
        # childnodes
        kids = [k for k in dpath[verb] if dpath[verb][k] == 1]
        for xcomp in xcomp_nodes:
            if xcomp not in kids:
                continue
            if not graph.nodes[xcomp]["feats"] or "VerbForm" not in graph.nodes[xcomp]["feats"].keys() or not graph.nodes[xcomp]["feats"]["VerbForm"] == 'Sup':
                continue
            
            #graph.draw_graph2(highlight=[verb, xcomp])
            d = {
                'id':  graph.get_metadata('row_nr'),
                'sentence':  graph.get_metadata('text'),
                'verb':  graph.nodes[verb]["lemma"],
                'xcomp':  graph.nodes[xcomp]["lemma"],
                'level':  graph.get_metadata('sent_level'),
                'sub': " ".join(
                            [graph.nodes[n]["form"] for n in sorted([verb] + kids)]
                        ),
            }
            
            collected_data.append(d)


../data/Eesti-keele-kui-teise-keele-kooliõpikute-lausete-korpus-2021.conllu
CPU times: user 4.07 s, sys: 41.6 ms, total: 4.11 s
Wall time: 4.12 s


In [10]:
df = pd.DataFrame.from_dict(collected_data)
df.to_csv(f'verb_xcompSup_{date_time}.csv', index=None)
df.head()

,id,sentence,verb,xcomp,level,sub
0,50,"Uusi kaaslasi oleks vaja tundma õppida, ent ei...",õppima,tundma,gümn,tundma õppida
1,394,"Teil tuleb õppida eristama neid sündmusi, mis ...",õppima,eristama,gümn,õppida eristama
2,395,Õppige neid hindama eesmärgi saavutamise seisu...,õppima,hindama,gümn,Õppige hindama seisukohalt .
3,397,Teil on kasulik õppida usaldama oma tundeid.,õppima,usaldama,gümn,õppida usaldama
4,416,Õpi mängima mõnd pilli.,õppima,mängima,gümn,Õpi mängima .
